In [ ]:
%reload_ext autoreload
%autoreload 2

import os

os.environ['NUMEXPR_MAX_THREADS'] = '20'
import logging
logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import neptune.new as neptune

from alphabase.spectral_library.flat import SpecLibFlat
from alphabase.spectral_library.base import SpecLibBase
from alphabase.spectral_library.reader import SWATHLibraryReader

from alphadia.extraction.data import TimsTOFDIA
from alphadia.extraction.planning import Plan, Workflow
from alphadia.extraction.calibration import RunCalibration
from alphadia.extraction.candidateselection import MS1CentricCandidateSelection
from alphadia.extraction.scoring import fdr_correction, unpack_fragment_info, MS2ExtractionWorkflow
import alphadia.extraction.utils as utils
yaml_file = 'config.yaml'

raw_files = ['/Users/georgwallmann/Documents/data/raw_data/synchro_PASEF_4prot/20220923_TIMS03_PaSk_SA_4prot_HeLa_Evo05_21min_IM0713_200fmol_classical_SyS_4MS_wCE_S4-C3_1_32275.d']

output_location = '/Users/georgwallmann/Documents/data/testing/2_feature_dev'

try:
    neptune_token = os.environ['NEPTUNE_TOKEN']
except KeyError:
    logging.error('NEPTUNE_TOKEN environtment variable not set')


In [ ]:
test_lib = SpecLibBase()
test_lib_location = '/Users/georgwallmann/Documents/data/raw_data/synchro_PASEF_4prot/dda_psms_maxquant/alpha_lib.hdf'
test_lib.load_hdf(test_lib_location, load_mod_seq=True)

plan = Plan(raw_files)
plan.from_spec_lib_base(test_lib)

for dia_data, precursors_flat, fragments_flat in plan.get_run_data():

    raw_name = precursors_flat.iloc[0]['raw_name']

    workflow = Workflow(
        plan.config, 
        dia_data, 
        precursors_flat, 
        fragments_flat
        )
    
    workflow.calibration()
    df = workflow.extraction(keep_decoys=True)

In [ ]:
fragments_flat

In [ ]:
from alphadia.extraction import candidateselection

selection = candidateselection.MS1CentricCandidateSelection(
    dia_data,
    precursors_flat,
    rt_tolerance=15,
    mz_tolerance=15,
    mobility_tolerance=0.02,
    candidate_count=3,
    thread_count=1,
    precursor_mz_column='mz_calibrated',
    rt_column='rt_calibrated',
    mobility_column='mobility_calibrated',
    debug=False
)

candidates = selection()

In [ ]:
from alphadia.extraction import quadrupole, scoring, features
q = quadrupole.SimpleQuadrupole(dia_data.cycle)

In [ ]:

extraction = scoring.MS2ExtractionWorkflow(
    dia_data,
    precursors_flat,
    fragments_flat,
    candidates,
    q,
    precursor_mz_tolerance=15,
    fragment_mz_tolerance=20,
    precursor_mz_column = 'mz_calibrated',
    fragment_mz_column = 'mz_calibrated',
    debug=False,
)

feature_df, fragment_df = extraction()


In [ ]:
print(feature_df.columns)

In [ ]:
feature_df['correlation_score'] = feature_df[['fragment_frame_correlation', 'fragment_scan_correlation', 'template_frame_correlation', 'template_scan_correlation']].apply(np.mean, axis=1)

In [ ]:
feature_df['precursor_idx'].nunique()
df = scoring.fdr_correction(feature_df, feature_columns=
            ['correlation_score'
            ]
            )

In [ ]:
sns.histplot(df, x='correlation_score', hue='decoy', stat='count',  bins=100, element='step')

In [ ]:
sns.scatterplot(data=df, x='fragment_frame_correlation', y='fragment_scan_correlation', hue='decoy')

In [ ]:
from alphadia.extraction import plotting, quadrupole
@nb.njit
def build_feature(
    dense_fragments,
    template,
    fragments

):

    total_fragment_intensity = np.sum(np.sum(dense_fragments[0], axis=-1), axis=-1)
    total_template_intensity = np.sum(np.sum(template, axis=-1), axis=-1)

    fragment_mask_2d = (total_fragment_intensity > 0).astype(np.int8)
    fragment_mask_1d = np.sum(fragment_mask_2d, axis=-1) > 0
    fragment_mask_2d = fragment_mask_2d * np.expand_dims(fragments.intensity, axis=-1)

    # (n_fragments, n_observations, n_frames)
    fragments_frame_profile = features.or_envelope_2d(features.frame_profile_2d(dense_fragments[0]))
    template_frame_profile = features.or_envelope_2d(features.frame_profile_2d(template))

    # (n_fragments, n_observations, n_scans)
    fragments_scan_profile = features.or_envelope_2d(features.scan_profile_2d(dense_fragments[0]))
    template_scan_profile = features.or_envelope_2d(features.scan_profile_2d(template))

    

    with nb.objmode:
        plotting.plot_fragment_profile(
            template,
            fragments_scan_profile,
            fragments_frame_profile,
            template_frame_profile,
            template_scan_profile,
        )
    

    # (n_fragments, n_observations)
    fragment_scan_correlation, template_scan_correlation = weighted_correlation(
        fragments_scan_profile,
        template_scan_profile,
        fragment_mask_2d,
    )

    
    # (n_fragments, n_observations)
    fragment_frame_correlation, template_frame_correlation = weighted_correlation(
        fragments_frame_profile,
        template_frame_profile,
        fragment_mask_2d,
    )

    observation_importance = quadrupole.calculate_observation_importance(template)
    weights = fragments.intensity / np.sum(fragments.intensity)

    fragment_scan_mean_list = np.sum(fragment_scan_correlation * observation_importance, axis = -1)
    fragment_scan_mean_agg = np.sum(fragment_scan_mean_list * weights)
    print(fragment_scan_mean_agg)

    fragment_frame_mean_list = np.sum(fragment_frame_correlation * observation_importance, axis = -1)
    fragment_frame_mean_agg = np.sum(fragment_frame_mean_list * weights)
    print(fragment_frame_mean_agg)

    template_scan_mean_list = np.sum(template_scan_correlation * observation_importance, axis = -1)
    template_scan_mean_agg = np.sum(template_scan_mean_list * weights)
    print(template_scan_mean_agg)

    template_frame_mean_list = np.sum(template_frame_correlation * observation_importance, axis = -1)
    template_frame_mean_agg = np.sum(template_frame_mean_list * weights)
    print(template_frame_mean_agg)


for candidate in candidate_container[:]:
    if len(candidate.template) > 0:
        res = build_feature(
            candidate.dense_fragments,
            candidate.template,
            candidate.fragments
        )
    

In [ ]:
arr = np.ones((5,5))
mask = np.ones((5,5))
np.fill_diagonal(mask,0)

features.weighted_mean_a1(arr, mask)

In [ ]:
fragment_profile

In [ ]:
dense_fragments = candidate_container[8].dense_fragments[0]
dense_fragments = dense_fragments

In [ ]:
template = candidate_container[8].template

In [ ]:
import numba as nb

@nb.guvectorize([(nb.float64[:,:], nb.float64[:])], '(n, k)->(k)')
def frame_profile(x, res):
    res[:] = np.sum(x, axis=0)

@nb.guvectorize([(nb.float64[:], nb.float64[:])], '(n)->()')
def scan_profile(x, res):
    res[0] = np.sum(x)

@nb.guvectorize([
    (nb.float64[:], nb.float64[:]),
    (nb.float32[:], nb.float32[:]),
    ], '(n)->(n)')
def or_envelope(x, res):
    res[:] = x
    for i in range(1, len(x) - 1):
        if (x[i] < x[i-1]) or (x[i] < x[i+1]):
            res[i] = (x[i-1] + x[i+1]) / 2

@nb.njit
def frame_profile_2d(x):
    return np.sum(x, axis=2)

@nb.njit
def frame_profile_1d(x):
    return np.sum(x, axis=1)

@nb.njit
def scan_profile_2d(x):
    return np.sum(x, axis=3)

@nb.njit
def scan_profile_1d(x):
    return np.sum(x, axis=2)

@nb.njit
def or_envelope_1d(x):
    res = x.copy()
    for a0 in range(x.shape[0]):
        for i in range(1, x.shape[1] - 1):
            if (x[a0, i] < x[a0, i-1]) or (x[a0, i] < x[a0, i+1]):
                res[a0, i] = (x[a0, i-1] + x[a0, i+1]) / 2

@nb.njit
def or_envelope_2d(x):
    res = x.copy()
    for a0  in range(x.shape[0]):
        for a1 in range(x.shape[1]):
            for i in range(1, x.shape[2] - 1):
                if (x[a0, a1, i] < x[a0, a1, i-1]) or (x[a0, a1, i] < x[a0, a1, i+1]):
                    res[a0, a1, i] = (x[a0, a1, i-1] + x[a0, a1, i+1]) / 2
    return res


In [ ]:
fragments_frame_profile = or_envelope_2d(frame_profile_2d(dense_fragments))
fragments_scan_profile = or_envelope_2d(scan_profile_2d(dense_fragments))

template_frame_profile = or_envelope_2d(frame_profile_2d(template))
template_scan_profile = or_envelope_2d(scan_profile_1d(template))

In [ ]:
fragments_scan_profile[:,i_observations].shape

In [ ]:
# (n_fragments, n_observations)
total_fragment_intensity = np.sum(np.sum(dense_fragments, axis=-1), axis=-1)
fragment_mask_2d = (total_fragment_intensity > 0).astype(np.int8)
fragment_mask_2d = fragment_mask_2d * np.expand_dims(candidate_container[8].fragments.intensity, axis=-1)


In [ ]:
fragment_mask_2d

In [ ]:






weighted_correlation(
    fragments_frame_profile,
    template_frame_profile,
    #np.ones_like(fragment_mask_2d),
    fragment_mask_2d,
)
#fragment_mask_2d.shape

In [ ]:
mask_2d = fragment_mask_2d[:,[0]]*fragment_mask_2d[:,0]
np.fill_diagonal(mask_2d, False)

In [ ]:
weighted_mean_a1(
    np.array([[1,2,np.nan],[4,5,6]]),
    np.array([[1,0,0],[1,1,1]])
)

In [ ]:

fragments_frame_profile = or_envelope(frame_profile(dense_fragments))
fragments_scan_profile = or_envelope(scan_profile(dense_fragments))

precursors_frame_profile = or_envelope(frame_profile(template))
precursors_scan_profile = or_envelope(scan_profile(template))

template_scan_profile = or_envelope(scan_profile(template))
template_frame_profile = or_envelope(frame_profile(template))

In [ ]:
fragments_frame_profile[:,0]

In [ ]:
@ nb.njit
def mean_correlation(
    fragment_profile,
    template_profile
):
    correlation = np.corrcoef(fragment_profile, template_profile)
    
    #mean_frame_corr = utils.amean0(correlation[:-1,:-1])-1/len(correlation[:-1,:-1])

    return correlation


In [ ]:
candidate_container[1].fragments.intensity

In [ ]:
plt.imshow(mean_correlation(fragments_frame_profile[:,0], template_frame_profile[0,0]))

In [ ]:
@nb.njit
def weighted_precursor_correlation(
    fragment_profile,
    weights,
):
    n_fragemts = fragment_profile.shape[0]
    n_observations = fragment_profile.shape[1]

    weights = weights / np.sum(weights)

    weighted_correlation = np.zeros((n_fragemts, n_observations))

    for i_observations in range(n_observations):
        correlation = np.corrcoef(fragments_frame_profile[:,i_observations])
        weighted_correlation[:,i_observations] = np.sum(correlation * weights, axis = 1)

    return weighted_correlation

In [ ]:
correlation

In [ ]:
correlation * weight

In [ ]:
weighted_correlation * 

In [ ]:
from alphadia.extraction import utils, plotting
correlations = utils.calculate_correlations(
        template_scan_profile[:,0], 
        fragments_scan_profile[:,0]
)

correlations

In [ ]:
scan_profile = np.sum(dense_fragments, axis = -1)
scan_profile.shape

In [ ]:
a = np.arange(20).reshape(20)
h(a)

In [ ]:
template_scan_profile = np.sum(template, axis=-1)
fragment_scan_profile = np.sum(dense_fragments, axis=-1)

In [ ]:
fragment_scan_profile.shape

In [ ]:

template_scan_profile.shape

In [ ]:
print(n_observations)

In [ ]:
feature_df['precursor_mass_error'] = np.abs(feature_df['precursor_mass_error'])
feature_df['rt_error'] = np.abs(feature_df['rt_error'])

In [ ]:
df = scoring.fdr_correction(feature_df)
df = scoring.fdr_correction(df)

In [ ]:
df_sig = df[df['qval'] <= 0.01]


In [ ]:
df.columns

In [ ]:
missing_ids = np.setdiff1d(candidates[candidates['decoy'] == 0]['precursor_idx'].unique(), ids)

In [ ]:
missing_ids

In [ ]:
from tqdm import tqdm
candidate_container = cc

fragment_collection = {'precursor_idx': []}
feature_collection = []

# initialize fragment collection with empty lists
for key in candidate_container[0].fragment_features.keys():
    fragment_collection[key] = []

for i, c in enumerate(tqdm(candidate_container)):

    n = 0
    for key, item in c.fragment_features.items():
        
        fragment_collection[key].append(item)
        n = len(item)
        
    fragment_collection['precursor_idx'].append(np.repeat(c.precursor_idx[0], n))
    

    if i > 10:
        break
    #feature_collection.append(self._collect_candidate(c))

for key, item in fragment_collection.items():
    fragment_collection[key] = np.concatenate(item)

In [ ]:
cc[0].precursor_idx[0]

In [ ]:
for key, item in fragment_collection.items():
    print(key, item.shape)

In [ ]:
collection_dict = {}

for key in cc[0].fragment_features.keys():
    collection_dict[key] = []

for c in cc:
    for key, item in c.fragment_features.items():
        collection_dict[key].append(item)

for key, item in cc[0].fragment_features.items():
    collection_dict[key] = np.concatenate(collection_dict[key])

df = pd.DataFrame(collection_dict)

In [ ]:
df

In [ ]:
import numba as nb

@nb.njit
def cosine_similarity_1d(template_intensity, fragments_intensity):

    fragment_norm = np.sqrt(np.sum(np.power(fragments_intensity,2),axis=-1))
    template_norm = np.sqrt(np.sum(np.power(template_intensity,2),axis=-1))

    div = (fragment_norm * template_norm) + 0.0001

    return np.sum(fragments_intensity * template_intensity,axis=-1) / div

In [ ]:
template_intensity = np.sum(np.sum(cc[0].template, axis=-1), axis=-1)
fragments_intensity = np.sum(np.sum(cc[0].dense_fragments[0], axis=-1), axis=-1)

fragment_mask_2d = fragments_intensity > 0
fragment_mask_1d = np.sum(fragment_mask_2d, axis=-1) > 0

print(fragment_mask_1d)

score = cosine_similarity_1d(template_intensity, fragments_intensity[fragment_mask_1d])

In [ ]:
score

In [ ]:
#template_dist = template_dist / np.sum(template_dist, axis=-1)
template_dist


In [ ]:

#fragments_dist = fragments_dist/np.sum(fragments_dist, axis=-1, keepdims=True)

fragments_dist

In [ ]:
fragments_flat[['mz_library']].values.astype(np.float32)

In [ ]:
dot = 
dot / 

In [ ]:
@nb.njit
def cosine_similarity_int(a, b):
    div = np.sqrt(np.sum(a))*np.sqrt(np.sum(b))
    if div == 0:
        return 0
    return np.sum((a*b))/div

In [ ]:
df.sort_values(by='base_width_mobility')[['base_width_mobility', 'precursor_idx','decoy']]

In [ ]:
df.sort_values('base_width_mobility')[['precursor_idx','elution_group_idx','base_width_mobility','decoy']].head(10)

In [ ]:
from alphadia.extraction import scoring
df = scoring.fdr_correction(df)


df['significant'] = df['qval'] <= 0.01

In [ ]:
df['mean_observation_score']

In [ ]:
df = df[df['n_observations'] == 1]
df[['mean_observation_score','precursor_idx','decoy']]

In [ ]:
df.columns

In [ ]:
sns.histplot(df, x='rt_error',hue='decoy', bins=100)

In [ ]:
sns.violinplot(data=df, y='mean_observation_score', x='n_observations', hue='decoy', split=True,bw=.15)

In [ ]:
sns.scatterplot(data=df, x='mean_observation_score', y='var_observation_score', hue='decoy')

In [ ]:
df_sig = df[df['qval'] < 0.05]
utils.density_scatter(df_sig['rt_library'].values, df_sig['rt_error'].values )

In [ ]:
import numba as nb



expand_cycle(dia_data.cycle,2)